In [34]:
import ollama
import fitz 
import pdfplumber
import json
from IPython.display import display
from PIL import Image


In [35]:

def convert_pdf_to_images(pdf_path):
    images = convert_from_path(pdf_path)
    return images


In [36]:

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text("text") + "\n"
    return text.strip()


In [37]:

def extract_tables_from_pdf(pdf_path):
    tables = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            extracted_tables = page.extract_tables()
            for table in extracted_tables:
                headers = table[0] 
                for row in table[1:]:
                    row_dict = {headers[i]: row[i] for i in range(len(headers))}
                    tables.append(row_dict)
    return tables


In [38]:

def structure_data_with_ollama(text, tables):
    prompt = f"""
    Extract structured data from this purchase order.
    - Headers: Extract all relevant key-value pairs.
    - list_items: Extract table data as structured lists.
    - Do not summarize or omit any data.
    - Add all list_items properly.

    Text:
    {text}

    Tables:
    {tables}

    Format response as:
    {{
        "headers": {{"key1": "value1", "key2": "value2"}},
        "list_items": [{{"column1": "value", "column2": "value"}}]
    }}
    """

    response = ollama.chat(model="mistral", messages=[{"role": "user", "content": prompt}])
    # response = ollama.chat(model="llama3", messages=[{"role": "user", "content": prompt}])

    return response['message']['content']


In [39]:

pdf_path = "Document Extraction 2.pdf" 
print(f"Processing {pdf_path}...")


Processing Document Extraction 2.pdf...


In [40]:

extracted_text = extract_text_from_pdf(pdf_path)
extracted_tables = extract_tables_from_pdf(pdf_path)


In [41]:

structured_json = structure_data_with_ollama(extracted_text, extracted_tables)


In [42]:
print("Raw Ollama Response:")
print(structured_json)  # Debugging: Check if response is valid JSON

Raw Ollama Response:
 Here is the structured data extracted from the provided purchase order:

    {
        "headers": {
            "DATE": "mm/dd/yyyy",
            "PURCHASE ORDER NO.": "8365234",
            "VENDOR INFORMATION.VENDOR NAME": "ABC Office Supplies",
            "VENDOR INFORMATION.ADDRESS": "1667 K Street NW, Washington DC 2006",
            "VENDOR INFORMATION.CONTACT NO.": "(203) 743-8993",
            "VENDOR INFORMATION.EMAIL ADDRESS": "<Email Address>",
            "CUSTOMER INFORMATION.CUSTOMER NAME": "Franklin Middle School",
            "CUSTOMER INFORMATION.ADDRESS": "Washington DC, USA",
            "CUSTOMER INFORMATION.CONTACT NO.": "(203) 334-7234",
            "CUSTOMER INFORMATION.EMAIL ADDRESS": "<Email Address>",
            "Item No.": None,
            "Details": None,
            "Unit": None,
            "Quantity": None,
            "Unit Price": None,
            "Total": None,
            "Taxes (%):": None,
            "Discount (%):": None,

In [43]:

with open("purchase_order_data.json", "w") as f:
    json.dump(structured_json, f, indent=4)


In [44]:

print("Purchase Order Data Extracted:")
print(structured_json)


Purchase Order Data Extracted:
 Here is the structured data extracted from the provided purchase order:

    {
        "headers": {
            "DATE": "mm/dd/yyyy",
            "PURCHASE ORDER NO.": "8365234",
            "VENDOR INFORMATION.VENDOR NAME": "ABC Office Supplies",
            "VENDOR INFORMATION.ADDRESS": "1667 K Street NW, Washington DC 2006",
            "VENDOR INFORMATION.CONTACT NO.": "(203) 743-8993",
            "VENDOR INFORMATION.EMAIL ADDRESS": "<Email Address>",
            "CUSTOMER INFORMATION.CUSTOMER NAME": "Franklin Middle School",
            "CUSTOMER INFORMATION.ADDRESS": "Washington DC, USA",
            "CUSTOMER INFORMATION.CONTACT NO.": "(203) 334-7234",
            "CUSTOMER INFORMATION.EMAIL ADDRESS": "<Email Address>",
            "Item No.": None,
            "Details": None,
            "Unit": None,
            "Quantity": None,
            "Unit Price": None,
            "Total": None,
            "Taxes (%):": None,
            "Discount (%